## Imports


In [49]:
import pandas as pd

### Qualitative analysis


#### Lloguer dataset

In [50]:
llogdf = pd.read_csv('2017_lloguer_preu_trim.csv')
llogdf.head()

,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
0,2017,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Lloguer mitjà mensual (Euros/mes),905.26
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Lloguer mitjà mensual (Euros/mes),722.78
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Lloguer mitjà mensual (Euros/mes),895.28
4,2017,1,2,Eixample,5,el Fort Pienc,Lloguer mitjà mensual (Euros/mes),871.08


In [51]:
llogdf.shape

(584, 8)

We have 584 rows and 8 columns. Now we will analyze the characteristics of the dataset.

#### Column investigation

In [52]:
# Writes relevant information about each column 
for col in llogdf.columns:
    print(
    f"""
    Column: {col}
    Type: {llogdf[col].dtype}
    Missing values: {llogdf[col].isna().sum()}    
    Number of unique values: {llogdf[col].nunique()}       
    """)


    Column: Any
    Type: int64
    Missing values: 0    
    Number of unique values: 1       
    

    Column: Trimestre
    Type: int64
    Missing values: 0    
    Number of unique values: 4       
    

    Column: Codi_Districte
    Type: int64
    Missing values: 0    
    Number of unique values: 10       
    

    Column: Nom_Districte
    Type: object
    Missing values: 0    
    Number of unique values: 10       
    

    Column: Codi_Barri
    Type: int64
    Missing values: 0    
    Number of unique values: 73       
    

    Column: Nom_Barri
    Type: object
    Missing values: 0    
    Number of unique values: 73       
    

    Column: Lloguer_mitja
    Type: object
    Missing values: 0    
    Number of unique values: 2       
    

    Column: Preu
    Type: float64
    Missing values: 38    
    Number of unique values: 504       
    


We can observe that the only column with Null values is `Preu`. We will deal with that later on. We can also see that we have 1 value for year and 4 values for `trimeste` (as we should have expected), 10 for `codi_districte` (and `nom_districte`), 73 for `codi_barri` (and `nom_barri`), 2 for `lloguer_mitjà`. Let us see which are these two values.

In [53]:
# show the unique values of "lloguer_mitja"
llogdf["Lloguer_mitja"].unique()

array(['Lloguer mitjà mensual (Euros/mes)',
       'Lloguer mitjà per superfície (Euros/m2 mes)'], dtype=object)

As we can see, it seems like we have the data of the price for each combination of district and neighborhood both in euros and euros per square meter.

#### Data reorganization

We will now reorganize the data in order to make analysis easier. The first step is to get rid of unnecesary columns such as "Any" (only one value) and "codi_districte" and "codi_barri" (we will use the names instead). We will also rename the columns to make them easier to use.

In [54]:
llogdf.drop(columns=["Any", "Codi_Barri", "Codi_Districte"], inplace=True)
llogdf.head()

,Trimestre,Nom_Districte,Nom_Barri,Lloguer_mitja,Preu
0,1,Ciutat Vella,el Raval,Lloguer mitjà mensual (Euros/mes),734.99
1,1,Ciutat Vella,el Barri Gòtic,Lloguer mitjà mensual (Euros/mes),905.26
2,1,Ciutat Vella,la Barceloneta,Lloguer mitjà mensual (Euros/mes),722.78
3,1,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",Lloguer mitjà mensual (Euros/mes),895.28
4,1,Eixample,el Fort Pienc,Lloguer mitjà mensual (Euros/mes),871.08


Now we will reduce the number of rows by grouping in the same row the data of the same district and neighborhood.

In [75]:
# rows where the price is the mean total price
llogdf_month = llogdf[llogdf["Lloguer_mitja"] == "Lloguer mitjà mensual (Euros/mes)"]


# rows where the price is the mean price per square meter
llogdf_sqrmet = llogdf[llogdf["Lloguer_mitja"] != "Lloguer mitjà mensual (Euros/mes)"]

llogdf_price = pd.merge(llogdf_month, llogdf_sqrmet, on=["Nom_Barri", "Nom_Districte", "Trimestre"],  how='inner')
llogdf_price.head()

,Trimestre,Nom_Districte,Nom_Barri,Lloguer_mitja_x,Preu_x,Lloguer_mitja_y,Preu_y
0,1,Ciutat Vella,el Raval,Lloguer mitjà mensual (Euros/mes),734.99,Lloguer mitjà per superfície (Euros/m2 mes),13.50
1,1,Ciutat Vella,el Barri Gòtic,Lloguer mitjà mensual (Euros/mes),905.26,Lloguer mitjà per superfície (Euros/m2 mes),14.14
2,1,Ciutat Vella,la Barceloneta,Lloguer mitjà mensual (Euros/mes),722.78,Lloguer mitjà per superfície (Euros/m2 mes),19.46
3,1,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",Lloguer mitjà mensual (Euros/mes),895.28,Lloguer mitjà per superfície (Euros/m2 mes),15.01
4,1,Eixample,el Fort Pienc,Lloguer mitjà mensual (Euros/mes),871.08,Lloguer mitjà per superfície (Euros/m2 mes),12.77


In [76]:
llogdf_price.shape

(292, 7)

We observe that the cardinality is what we expected. Now 